In [ ]:
from glider import glider
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 100
x_opt, u_opt, opt_guess, sol = glider(N=N)

In [ ]:
np.save("optimal_sol", x_opt)
np.save("optimal_u", u_opt)

In [ ]:
plt.plot(x_opt[0])
plt.plot(x_opt[1])
plt.legend(["u", "v"])
plt.show()

In [ ]:
plt.plot(x_opt[3], x_opt[4])
plt.title("Trajectory")
plt.show()

In [ ]:
plt.plot(x_opt[6])
plt.title("Beta")
plt.show()

In [ ]:
x_opt[3]

In [ ]:
x_opt[4]

In [ ]:
opt_guess

In [ ]:
import pickle

# Open a file and use dump()
with open("file.pkl", "wb") as file:
    # A new file will be created
    pickle.dump(opt_guess, file)

In [ ]:
# Open the file in binary mode
with open("file.pkl", "rb") as file:
    # Call load method to deserialze
    opt_guess = pickle.load(file)

In [ ]:
def construct_initial_guess(
    x_opt: np.ndarray, u_opt: np.ndarray, N: int, p0: float
) -> list:
    """
    Construct an initial guess for the new N
    """
    w0 = []
    for i in range(N + 1):
        state = list(x_opt[:, i])
        if i == 0:
            w0.append([p0])
            w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
        elif i == N:
            for j in range(4):
                w0.append(state)
        elif i > 0 and i < N:
            for j in range(4):
                w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
    return w0

In [ ]:
w0 = construct_initial_guess(x_opt, u_opt, N=N, p0=17)
w0_arr = np.concatenate(w0)

In [ ]:
np.save("w0", w0_arr)

In [ ]:
# This code takes a solution from a simulation with N control steps and
# upsamples it so it can be used as an intial guess for a simulation with 2*N
# control steps
w_new = []
w_new.append(w0[0])
w_new.append(w0[1])
new_N = 2 * N
for i in range(N):
    for k in range(2):
        w_new.append([u_opt[0, i]])
        for j in range(4):
            state = list(x_opt[:, i + 1])
            w_new.append(state)

In [ ]:
w_new_arr = np.concatenate(w_new)

In [ ]:
np.save("double_w0", w_new_arr)

In [ ]:
plt.plot(u_opt[0])